# PIP installs

In [2]:
!pip install rasterio
!pip install geopandas
!pip install rioxarray
!pip install geojson
!pip install shapely
!pip install tqdm

     |████████████████████████████████| 19.3 MB 1.3 MB/s 
     |████████████████████████████████| 994 kB 7.6 MB/s 
     |████████████████████████████████| 6.3 MB 22.2 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 46 kB 2.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rioxarray: filename=rioxarray-0.7.1-py3-none-any.whl size=54118 sha256=83e527c08830b6b5dfd2bc3167559136274b594bf866827d0dd26dfd1281c190
  Stored in directory: /root/.cache/pip/wheels/42/9b/1c/acf2904f7ee44be3c0b8ffc7ea4fafebb135aee00242bef86f
Successfully built rioxarray


In [3]:
!pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Imports


In [34]:
import rioxarray
import geopandas as gpd
import pyproj
from shapely.geometry.point import Point
from shapely.geometry.polygon import Polygon
import geojson
import tqdm
import pandas as pd

# File paths

In [35]:
tif_file = '/content/drive/MyDrive/spacehackathon/GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0_22_5.tif'
parquet_file_path = '/content/drive/MyDrive/spacehackathon/data_coordinates.gzip'
districts_data_path = '/content/drive/MyDrive/spacehackathon/districts.geojson'
dammam_districts_data_path = 'dammam_districts.geojson'
dammam_districts_population_data_path = "dammam_districts_populations.json"

# Importing datasets

In [36]:
dmm_districts_df = gpd.read_file(dammam_districts_data_path)

In [37]:
dmm_districts_df

,district_id,city_id,region_id,name_ar,name_en,geometry
0,10500013001,13,5,حي الخالدية الشمالية,Al Khalidiyah Ash Shamaliyah Dist.,"POLYGON ((50.13178 26.42027, 50.13176 26.42027..."
1,10500013002,13,5,حي الناصرية,An Nasriyah Dist.,"POLYGON ((50.13004 26.42473, 50.12938 26.42498..."
2,10500013003,13,5,حي الصناعية الاولى,1St Industrial Dist.,"POLYGON ((50.15106 26.39182, 50.15116 26.39219..."
3,10500013004,13,5,حي الفنار,Al Fanar Dist.,"POLYGON ((50.21842 26.41224, 50.22039 26.41322..."
4,10500013005,13,5,حي الأثير,Al Athir Dist.,"POLYGON ((50.06185 26.42225, 50.06214 26.42241..."
...,...,...,...,...,...,...
78,10500013080,13,5,حي الفرسان,Al Fursan Dist.,"POLYGON ((49.95704 26.33672, 49.95705 26.33663..."
79,10500013081,13,5,حي المها,Al Maha Dist.,"POLYGON ((49.94447 26.50012, 49.94420 26.50027..."
80,10500013082,13,5,حي الندى,An Nada Dist.,"POLYGON ((50.09052 26.37495, 50.08177 26.38848..."
81,10500013083,13,5,المدينة الصناعية الثالثة,3Rd Industrial City,"POLYGON ((50.01507 25.91677, 50.00782 25.93476..."


# Helper methods

Writing dataframe to desk to avoid re-computation when kernel dies.

# Districts Coordinates

Narrow down dataset to Dammam

# Filtering to Dammam

In [38]:
import pandas as pd
api_df = pd.DataFrame()

In [39]:
import numpy as np
api_df['district_id'] = np.nan
api_df['population_density'] = np.nan
api_df['connectivity'] = np.nan
api_df['services'] = np.nan

In [40]:
api_df

,district_id,population_density,connectivity,services


In [41]:
pop_df = pd.read_json(dammam_districts_population_data_path)

In [42]:
pop_df

,district_id,population
0,10500013001,40247
1,10500013002,18579
2,10500013003,16426
3,10500013004,2513
4,10500013005,22234
...,...,...
78,10500013080,4318
79,10500013081,48
80,10500013082,27080
81,10500013083,51


In [43]:
pop_df.describe()

,district_id,population
count,8.300000e+01,83.000000
mean,1.050001e+10,17812.084337
std,2.438950e+01,14535.207265
min,1.050001e+10,0.000000
25%,1.050001e+10,5969.000000
50%,1.050001e+10,14266.000000
75%,1.050001e+10,26812.500000
max,1.050001e+10,59380.000000


In [44]:
dmm_districts_df["area_km"] = dmm_districts_df['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [45]:
dmm_districts_df

,district_id,city_id,region_id,name_ar,name_en,geometry,area_km
0,10500013001,13,5,حي الخالدية الشمالية,Al Khalidiyah Ash Shamaliyah Dist.,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",7.844822
1,10500013002,13,5,حي الناصرية,An Nasriyah Dist.,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",1.625616
2,10500013003,13,5,حي الصناعية الاولى,1St Industrial Dist.,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",4.610420
3,10500013004,13,5,حي الفنار,Al Fanar Dist.,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",5.122655
4,10500013005,13,5,حي الأثير,Al Athir Dist.,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",2.854831
...,...,...,...,...,...,...,...
78,10500013080,13,5,حي الفرسان,Al Fursan Dist.,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",5.950598
79,10500013081,13,5,حي المها,Al Maha Dist.,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",7.838438
80,10500013082,13,5,حي الندى,An Nada Dist.,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",5.025529
81,10500013083,13,5,المدينة الصناعية الثالثة,3Rd Industrial City,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",72.769348


In [46]:
pop_aread_df = pop_df.join(dmm_districts_df.set_index('district_id'), on='district_id')

In [47]:
pop_aread_df

,district_id,population,city_id,region_id,name_ar,name_en,geometry,area_km
0,10500013001,40247,13,5,حي الخالدية الشمالية,Al Khalidiyah Ash Shamaliyah Dist.,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",7.844822
1,10500013002,18579,13,5,حي الناصرية,An Nasriyah Dist.,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",1.625616
2,10500013003,16426,13,5,حي الصناعية الاولى,1St Industrial Dist.,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",4.610420
3,10500013004,2513,13,5,حي الفنار,Al Fanar Dist.,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",5.122655
4,10500013005,22234,13,5,حي الأثير,Al Athir Dist.,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",2.854831
...,...,...,...,...,...,...,...,...
78,10500013080,4318,13,5,حي الفرسان,Al Fursan Dist.,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",5.950598
79,10500013081,48,13,5,حي المها,Al Maha Dist.,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",7.838438
80,10500013082,27080,13,5,حي الندى,An Nada Dist.,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",5.025529
81,10500013083,51,13,5,المدينة الصناعية الثالثة,3Rd Industrial City,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",72.769348


In [48]:
pop_aread_df['pop_density'] = pop_aread_df['population'] / pop_aread_df['area_km']

In [49]:
pop_area_df = pop_aread_df

In [50]:
pop_area_df

,district_id,population,city_id,region_id,name_ar,name_en,geometry,area_km,pop_density
0,10500013001,40247,13,5,حي الخالدية الشمالية,Al Khalidiyah Ash Shamaliyah Dist.,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",7.844822,5130.390542
1,10500013002,18579,13,5,حي الناصرية,An Nasriyah Dist.,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",1.625616,11428.900708
2,10500013003,16426,13,5,حي الصناعية الاولى,1St Industrial Dist.,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",4.610420,3562.799396
3,10500013004,2513,13,5,حي الفنار,Al Fanar Dist.,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",5.122655,490.565945
4,10500013005,22234,13,5,حي الأثير,Al Athir Dist.,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",2.854831,7788.201555
...,...,...,...,...,...,...,...,...,...
78,10500013080,4318,13,5,حي الفرسان,Al Fursan Dist.,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",5.950598,725.641404
79,10500013081,48,13,5,حي المها,Al Maha Dist.,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",7.838438,6.123669
80,10500013082,27080,13,5,حي الندى,An Nada Dist.,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",5.025529,5388.487575
81,10500013083,51,13,5,المدينة الصناعية الثالثة,3Rd Industrial City,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",72.769348,0.700845


In [55]:
pop_area_df['pop_density_index'] = ((pop_area_df['pop_density']-pop_area_df['pop_density'].min())/(pop_area_df['pop_density'].max() - pop_area_df['pop_density'].min())) * 100

In [56]:
pop_area_df

,district_id,population,city_id,region_id,name_ar,name_en,geometry,area_km,pop_density,pop_density_normalized,pop_density_index
0,10500013001,40247,13,5,حي الخالدية الشمالية,Al Khalidiyah Ash Shamaliyah Dist.,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",7.844822,5130.390542,32.658798,32.658798
1,10500013002,18579,13,5,حي الناصرية,An Nasriyah Dist.,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",1.625616,11428.900708,72.753557,72.753557
2,10500013003,16426,13,5,حي الصناعية الاولى,1St Industrial Dist.,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",4.610420,3562.799396,22.679900,22.679900
3,10500013004,2513,13,5,حي الفنار,Al Fanar Dist.,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",5.122655,490.565945,3.122822,3.122822
4,10500013005,22234,13,5,حي الأثير,Al Athir Dist.,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",2.854831,7788.201555,49.577766,49.577766
...,...,...,...,...,...,...,...,...,...,...,...
78,10500013080,4318,13,5,حي الفرسان,Al Fursan Dist.,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",5.950598,725.641404,4.619254,4.619254
79,10500013081,48,13,5,حي المها,Al Maha Dist.,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",7.838438,6.123669,0.038982,0.038982
80,10500013082,27080,13,5,حي الندى,An Nada Dist.,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",5.025529,5388.487575,34.301780,34.301780
81,10500013083,51,13,5,المدينة الصناعية الثالثة,3Rd Industrial City,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",72.769348,0.700845,0.004461,0.004461


----------------------------------

TODO:


1.   Fetch remaining JSON Data (internet coverage, hospitals, mosques, schools, phrarmacies, etc.)
2.   {Normalize} them (range 0-100)
3. Add together in single df, then save as json with columns {district_id, name_en, name_ar, geometry, pop_den_index, fast_internet_index, cellular_coverage_index, public_services_index, electricity_index, water_index}
4. Integrate with front-end

* fast internet = fiber + 5g
* ceullar = 2g, 3g, 4g

In [68]:
# Read JSONs
dmm_hos_df = pd.read_json('dammam_hospitals_per_district.json')
dmm_mos_df = pd.read_json('dammam_mosques_per_district.json')
dmm_pha_df = pd.read_json('dammam_pharmacies_per_district.json')
dmm_sch_df = pd.read_json('dammam_schools_per_district.json')
fiber_df = pd.read_json('fiber_coverage_per_district.json')
mobily_df = pd.read_json('mobily_cellular_coverage_district.json')
zain_df = pd.read_json('zain_5g_district_coverage.json')
aqar_df = pd.read_json('districts_with_aqar_ratings.json')
dmm_dist_df = gpd.read_file('dammam_districts.geojson')

In [69]:
indicators_df = dmm_dist_df[['district_id','name_en','name_ar','geometry']].copy()
# Public Services Indicator prework
dmm_hos_df = dmm_hos_df.rename(columns = {'count':'count_hos'})
dmm_mos_df = dmm_mos_df.rename(columns = {'number_of_mosques':'count_mos'})
dmm_pha_df = dmm_pha_df.rename(columns = {'count':'count_pha'})
dmm_sch_df = dmm_sch_df.rename(columns = {'count':'count_sch'})
# mobily_df_cellular = mobily_df[(mobily_df['coverage_type'] == '2g') | (mobily_df['coverage_type'] == '3g') | (mobily_df['coverage_type'] == '4g')]
mobily_df_fast = mobily_df[(mobily_df['coverage_type'] == '5g')]
mobily_df_fast['coverage_normalized_mobily'] = ((mobily_df_fast['coverage']-mobily_df_fast['coverage'].min())/(mobily_df_fast['coverage'].max() - mobily_df_fast['coverage'].min())) * 100
del mobily_df_fast['coverage']
del mobily_df_fast['coverage_type']
fiber_df['coverage_normalized_fiber'] = ((fiber_df['coverage']-fiber_df['coverage'].min())/(fiber_df['coverage'].max() - fiber_df['coverage'].min())) * 100
del fiber_df['coverage']
zain_df['coverage_normalized_zain'] = ((zain_df['coverage']-zain_df['coverage'].min())/(zain_df['coverage'].max() - zain_df['coverage'].min())) * 100
del zain_df['coverage']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [70]:
# Fast Internet Indicator
fast_internet_df = pd.DataFrame()
#fast_internet_df = zain_df.join(fiber_df.set_index('district_id'), on='district_id')
fast_internet_df = zain_df.merge(mobily_df_fast,on='district_id').merge(fiber_df,on='district_id')
fast_internet_df = fast_internet_df.fillna(0)
fast_internet_df['fast_internet'] = fast_internet_df['coverage_normalized_fiber']*0.5 + fast_internet_df['coverage_normalized_mobily']*0.25 + fast_internet_df['coverage_normalized_zain']*0.25
fast_internet_df['fast_internet_index'] = ((fast_internet_df['fast_internet']-fast_internet_df['fast_internet'].min())/(fast_internet_df['fast_internet'].max() - fast_internet_df['fast_internet'].min())) * 100
del fast_internet_df['coverage_normalized_zain']
del fast_internet_df['coverage_normalized_mobily']
del fast_internet_df['coverage_normalized_fiber']
del fast_internet_df['fast_internet']
fast_internet_df


,district_id,fast_internet_index
0,10100003001,59.606195
1,10100003002,51.542269
2,10100003003,25.729284
3,10100003004,23.379980
4,10100003006,30.398129
...,...,...
874,11302237028,27.426414
875,11302237032,44.446484
876,11302237054,26.708799
877,11302237055,23.195594


In [71]:
indicators_df = indicators_df.merge(fast_internet_df, on='district_id', how='left')
indicators_df = indicators_df.fillna(0)
indicators_df

,district_id,name_en,name_ar,geometry,fast_internet_index
0,10500013001,Al Khalidiyah Ash Shamaliyah Dist.,حي الخالدية الشمالية,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",0.000000
1,10500013002,An Nasriyah Dist.,حي الناصرية,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",65.747931
2,10500013003,1St Industrial Dist.,حي الصناعية الاولى,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",22.951775
3,10500013004,Al Fanar Dist.,حي الفنار,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",0.000000
4,10500013005,Al Athir Dist.,حي الأثير,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",46.333395
...,...,...,...,...,...
78,10500013080,Al Fursan Dist.,حي الفرسان,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",0.000000
79,10500013081,Al Maha Dist.,حي المها,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",0.000000
80,10500013082,An Nada Dist.,حي الندى,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",53.250593
81,10500013083,3Rd Industrial City,المدينة الصناعية الثالثة,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",0.000000


In [72]:
# Infras.. Indicator
# infrastructure_df = aqar_df[['district_id']].copy()
infrastructure_df = dmm_dist_df[['district_id']]
infrastructure_df['infrastructure_index'] = aqar_df['aqar_electricity_water_rate'] * 20 * 0.6 + aqar_df['aqar_streets_rate'] * 20 * 0.4
infrastructure_df
indicators_df = indicators_df.merge(infrastructure_df, on='district_id', how='left')
indicators_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,district_id,name_en,name_ar,geometry,fast_internet_index,infrastructure_index
0,10500013001,Al Khalidiyah Ash Shamaliyah Dist.,حي الخالدية الشمالية,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",0.000000,67.32596
1,10500013002,An Nasriyah Dist.,حي الناصرية,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",65.747931,76.82796
2,10500013003,1St Industrial Dist.,حي الصناعية الاولى,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",22.951775,62.23996
3,10500013004,Al Fanar Dist.,حي الفنار,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",0.000000,58.58400
4,10500013005,Al Athir Dist.,حي الأثير,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",46.333395,59.34204
...,...,...,...,...,...,...
78,10500013080,Al Fursan Dist.,حي الفرسان,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",0.000000,76.26300
79,10500013081,Al Maha Dist.,حي المها,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",0.000000,85.91300
80,10500013082,An Nada Dist.,حي الندى,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",53.250593,80.28100
81,10500013083,3Rd Industrial City,المدينة الصناعية الثالثة,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",0.000000,80.34796


In [76]:
# Public Services Indicator
public_services_df = dmm_dist_df[['district_id']]
public_services_df = public_services_df.merge(dmm_hos_df,on='district_id', how='left').merge(dmm_mos_df,on='district_id', how='left').merge(dmm_pha_df,on='district_id', how='left').merge(dmm_sch_df,on='district_id', how='left')
public_services_df = public_services_df.fillna(0)
public_services_df["hospitals_index"] =  100 * public_services_df['count_hos'] / pop_area_df["population"]
public_services_df["hospitals_index"] = ((public_services_df['hospitals_index']-public_services_df['hospitals_index'].min())/(public_services_df['hospitals_index'].max() - public_services_df['hospitals_index'].min())) * 100
public_services_df["pharmacies_index"] =  100 * public_services_df['count_pha'] / pop_area_df["population"]
public_services_df["pharmacies_index"] = ((public_services_df['pharmacies_index']-public_services_df['pharmacies_index'].min())/(public_services_df['pharmacies_index'].max() - public_services_df['pharmacies_index'].min())) * 100

public_services_df["mosques_index"] =  100 * public_services_df['count_mos'] / pop_area_df["population"]
public_services_df["mosques_index"] = ((public_services_df['mosques_index']-public_services_df['mosques_index'].min())/(public_services_df['mosques_index'].max() - public_services_df['mosques_index'].min())) * 100
public_services_df["schools_index"] =  100 * public_services_df['count_sch'] / pop_area_df["population"]
public_services_df["schools_index"] = ((public_services_df['schools_index']-public_services_df['schools_index'].min())/(public_services_df['schools_index'].max() - public_services_df['schools_index'].min())) * 100

del public_services_df['count_hos']
del public_services_df['count_mos']
del public_services_df['count_pha']
del public_services_df['count_sch']
public_services_df.describe()

,district_id,hospitals_index,pharmacies_index,mosques_index,schools_index
count,8.300000e+01,82.000000,82.000000,82.000000,82.000000
mean,1.050001e+10,6.342053,14.602915,16.586654,13.020970
std,2.438950e+01,14.141943,19.739942,24.715924,17.048838
min,1.050001e+10,0.000000,0.000000,0.000000,0.000000
25%,1.050001e+10,0.000000,0.000000,0.000000,0.000000
50%,1.050001e+10,0.000000,7.469064,3.407582,6.905326
75%,1.050001e+10,6.700952,18.615651,25.175277,19.587545
max,1.050001e+10,100.000000,100.000000,100.000000,100.000000


In [77]:
indicators_df = indicators_df.merge(public_services_df, on='district_id', how='left')
indicators_df

,district_id,name_en,name_ar,geometry,fast_internet_index,infrastructure_index,hospitals_index,pharmacies_index,mosques_index,schools_index
0,10500013001,Al Khalidiyah Ash Shamaliyah Dist.,حي الخالدية الشمالية,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",0.000000,67.32596,0.000000,1.925800,3.836311,0.000000
1,10500013002,An Nasriyah Dist.,حي الناصرية,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",65.747931,76.82796,0.000000,0.000000,12.465687,10.684875
2,10500013003,1St Industrial Dist.,حي الصناعية الاولى,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",22.951775,62.23996,5.263535,0.000000,4.699866,0.000000
3,10500013004,Al Fanar Dist.,حي الفنار,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",0.000000,58.58400,0.000000,0.000000,0.000000,0.000000
4,10500013005,Al Athir Dist.,حي الأثير,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",46.333395,59.34204,0.000000,6.971997,3.472160,0.000000
...,...,...,...,...,...,...,...,...,...,...
78,10500013080,Al Fursan Dist.,حي الفرسان,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",0.000000,76.26300,0.000000,35.899811,0.000000,0.000000
79,10500013081,Al Maha Dist.,حي المها,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",0.000000,85.91300,0.000000,0.000000,0.000000,0.000000
80,10500013082,An Nada Dist.,حي الندى,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",53.250593,80.28100,0.000000,11.448699,0.000000,1.221777
81,10500013083,3Rd Industrial City,المدينة الصناعية الثالثة,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",0.000000,80.34796,0.000000,0.000000,0.000000,0.000000


count    82.000000
mean      0.007335
std       0.016357
min       0.000000
25%       0.000000
50%       0.000000
75%       0.007750
max       0.115662
Name: hospitals_index, dtype: float64

In [78]:
pop_area_df = pop_area_df.rename(columns = {'district_ids' : 'district_id', 'pop_density_normalized': 'population_density_index'})
indicators_df = pd.merge(indicators_df,pop_area_df[['district_id','population_density_index']],on='district_id', how='left')
indicators_df = indicators_df.fillna(0)
indicators_df

,district_id,name_en,name_ar,geometry,fast_internet_index,infrastructure_index,hospitals_index,pharmacies_index,mosques_index,schools_index,population_density_index
0,10500013001,Al Khalidiyah Ash Shamaliyah Dist.,حي الخالدية الشمالية,"POLYGON ((50.13178 26.42027, 50.13176 26.42027...",0.000000,67.32596,0.000000,1.925800,3.836311,0.000000,32.658798
1,10500013002,An Nasriyah Dist.,حي الناصرية,"POLYGON ((50.13004 26.42473, 50.12938 26.42498...",65.747931,76.82796,0.000000,0.000000,12.465687,10.684875,72.753557
2,10500013003,1St Industrial Dist.,حي الصناعية الاولى,"POLYGON ((50.15106 26.39182, 50.15116 26.39219...",22.951775,62.23996,5.263535,0.000000,4.699866,0.000000,22.679900
3,10500013004,Al Fanar Dist.,حي الفنار,"POLYGON ((50.21842 26.41224, 50.22039 26.41322...",0.000000,58.58400,0.000000,0.000000,0.000000,0.000000,3.122822
4,10500013005,Al Athir Dist.,حي الأثير,"POLYGON ((50.06185 26.42225, 50.06214 26.42241...",46.333395,59.34204,0.000000,6.971997,3.472160,0.000000,49.577766
...,...,...,...,...,...,...,...,...,...,...,...
78,10500013080,Al Fursan Dist.,حي الفرسان,"POLYGON ((49.95704 26.33672, 49.95705 26.33663...",0.000000,76.26300,0.000000,35.899811,0.000000,0.000000,4.619254
79,10500013081,Al Maha Dist.,حي المها,"POLYGON ((49.94447 26.50012, 49.94420 26.50027...",0.000000,85.91300,0.000000,0.000000,0.000000,0.000000,0.038982
80,10500013082,An Nada Dist.,حي الندى,"POLYGON ((50.09052 26.37495, 50.08177 26.38848...",53.250593,80.28100,0.000000,11.448699,0.000000,1.221777,34.301780
81,10500013083,3Rd Industrial City,المدينة الصناعية الثالثة,"POLYGON ((50.01507 25.91677, 50.00782 25.93476...",0.000000,80.34796,0.000000,0.000000,0.000000,0.000000,0.004461


In [79]:
indicators_df.to_file('dataframe.geojson', driver='GeoJSON')  
